# Lab 1

In [ ]:
# Code Block 1
cw.plot(cap_pass_trace("h\n")) * cw.plot(cap_pass_trace("0\n"))

In [ ]:
# Code Block 2
from tqdm.notebook import tqdm
fig = cw.plot()
for c in tqdm('abcdefghijklmnopqrstuvwxyz0123456789'):
    trace = cap_pass_trace(c + "\n")
    fig *= cw.plot(trace[0:1000])
    
display(fig)

In [ ]:
# Code Block 3
ref_trace = cap_pass_trace("h0p\x00\n")[0:1000]
plot = cw.plot()
for c in 'abcdefghijklmnopqrstuvwxyz0123456789': 
    trace = cap_pass_trace('h0p' + c + "\n")[0:1000]
    plot *= cw.plot(trace - ref_trace)
display(plot)

In [ ]:
# Code Block 4
ref_trace = cap_pass_trace( "h0p\x00\n")


for c in 'abcdefghijklmnopqrstuvwxyz0123456789': 
    trace = cap_pass_trace("h0p" + c + "\n")
    diff = np.sum(np.abs(trace - ref_trace))
    
    print("{:1} diff = {:2}".format(c, diff))

In [ ]:
guessed_pw = ""

for _ in range(0, 5):  
    biggest_diff = 0
    biggest_char = '\x00'
    ref_trace = cap_pass_trace(guessed_pw + "\x00\n")
    
    for c in 'abcdefghijklmnopqrstuvwxyz0123456789': 
        trace = cap_pass_trace(guessed_pw + c + "\n")
        diff = np.sum(np.abs(trace - ref_trace))

        if diff > biggest_diff:
            biggest_diff = diff
            biggest_char = c
            
    guessed_pw += biggest_char
    print(guessed_pw)